In [1]:
import sys
sys.path.append("../")

import jax
import jax.numpy as np
from functools import partial
import jraph
from jax_md import space, partition

from models.graph_utils import RadiusSearch

In [6]:
n_batch = 4
n_nodes = 5000
n_pos = 3

x = np.load("/n/holyscratch01/iaifi_lab/ccuesta/data_for_sid/halos.npy")[:n_batch, :n_nodes]
x_pos = x[:, :, :n_pos]

In [7]:
ns = RadiusSearch(box_size=1000., cutoff=30.)
nbr = ns.init_neighbor_lst(x_pos[0])

In [9]:
nbr_update, nbr = ns.update_neighbor_lst(x_pos, nbr)

In [21]:
rng = jax.random.PRNGKey(42)
globals = jax.random.normal(rng, (n_batch, 1, 2))

In [22]:
graph = jax.vmap(partition.to_jraph)(nbr_update, mask=None,  nodes=x, edges=None, globals=globals)

In [23]:
graph.n_node.shape, graph.n_edge.shape, graph.nodes.shape, graph.globals.shape, graph.senders.shape, graph.receivers.shape, graph.receivers.shape 

((4, 2), (4, 2), (4, 5001, 7), (4, 2, 2), (4, 29868), (4, 29868), (4, 29868))

In [24]:
from models.gnn import GraphConvNet
import flax.linen as nn

class GraphWrapper(nn.Module):
    @nn.compact
    def __call__(self, x):
        model = jax.vmap(GraphConvNet(
            latent_size=6, 
            num_mlp_layers=4, 
            message_passing_steps=3))
        return model(x)

In [25]:
model = GraphWrapper()
graph_update, params  = model.init_with_output(rng, graph)

ValueError: Incompatible shapes for broadcasting: (2,) and requested shape (1,)